In [169]:
import pandas as pd
import numpy as np
import mysql.connector
import os
from mysql.connector import Error
from datetime import datetime, date, timedelta
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [170]:
#Definimos la ruta en donde queremos guardar los archivos
mes2 = 'Enero2021'
ruta_guardar = '/home/estadistico/Documents/Erick/'+mes2+'/Reportes diarios/Avance pagos'
ruta_pagos = '/home/estadistico/Documents/Erick'
mes = 202010101
mes3 = pd.to_datetime('01/01/2021',dayfirst=True)
prim_sem_mes = 1

In [171]:
quetzal = 2.54
dolar = 19.8

In [172]:
#Asignamos valores a los parametros"
today = date.today().strftime('%Y%m%d')
now = datetime.now().strftime('%d-%m-%Y %H-%M')
hoy = np.datetime64('today')
ayer = np.datetime64(date.today() - timedelta(1))
today2 = date.today().strftime('%d-%m-%Y')

# Torreon

In [173]:
metas = pd.DataFrame([('Avon',990000),('Oriflame',65000),('Invex_RLN',270000),('Invex_RLA',479000),('Invex_REL',470000),('Arabela',95000),('Cristian Lay',6000)])
metas.columns = ['Cliente','Meta']

In [174]:
#Lectura pagos Oriflam
#pagos_oriflame = pd.read_excel(''+ruta_pagos+'/'+mes2+'/Torreon/Oriflame/RECUPERACION EXITOSA 31-12-2020.xlsx')
pagos_oriflame = pd.read_excel(''+ruta_pagos+'/'+mes2+'/Torreon/Oriflame/Pagos/RECUPERACION EXITOSA 12-01-2021.xlsx')
pagos_oriflame['Cliente'] = 'Oriflame'
pagos_oriflame = pagos_oriflame.drop_duplicates()
pagos_oriflame['Factura'] = pagos_oriflame['PAGO_TOTAL']*.1
# pagos_oriflame = pagos_oriflame.loc[pagos_oriflame['FECHA_PAGO']>=mes3]

In [175]:
#Lectura pagos Arabela
# pagos_arabela = pd.read_excel(''+ruta_pagos+'/'+mes2+'/Torreon/Arabela/31122020_PAGOS_ARABELA.xlsx')
pagos_arabela = pd.read_excel(''+ruta_pagos+'/'+mes2+'/Torreon/Arabela/Pagos/PAGOS_ARABELA_12012021.xlsx')
pagos_arabela['Cliente'] = 'Arabela'
pagos_arabela = pagos_arabela.drop_duplicates()
pagos_arabela['FechaEntrega'] = [datetime.strptime(str(x), '%Y%m%d').strftime('%d/%m/%Y') for x in pagos_arabela['FechaEntrega']]
pagos_arabela['FechaEntrega'] = pd.to_datetime(pagos_arabela['FechaEntrega'],dayfirst=True)
pagos_arabela = pagos_arabela.loc[pagos_arabela['FechaEntrega']>=mes3]
pagos_arabela['factura'] = pagos_arabela['pago']*.22

In [176]:
#Lectura pagos invex
# pagos_invex = pd.read_excel(''+ruta_pagos+'/'+mes2+'/Torreon/Invex/Pagos Invex.xlsx')
pagos_invex = pd.read_excel(''+ruta_pagos+'/'+mes2+'/Torreon/Invex/Pagos/Pagos Invex '+mes2+'.xlsx')
pagos_invex['Cliente'] = 'Invex'
pagos_invex = pagos_invex.drop_duplicates()
pagos_invex['TRAN_POST_DT'] = [datetime.fromordinal(datetime(1900, 1, 1).toordinal() + x - 2) for x in pagos_invex['TRAN_POST_DT']]
# pagos_invex = pagos_invex.loc[pagos_invex['TRAN_POST_DT']>=mes3]

# Oriflame

In [177]:
reporte_pagos_oriflame = pd.pivot_table(pagos_oriflame,index=['Cliente'],values=['Factura'],aggfunc=np.sum)
reporte_pagos_oriflame = pd.DataFrame(reporte_pagos_oriflame.to_records())
reporte_pagos_oriflame.columns = ['Cliente','Facturacion']
reporte_pagos_oriflame['FechaFact'] = max(pagos_oriflame['FECHA_PAGO'])
reporte_pagos_oriflame['FechaFact'] = pd.to_datetime(reporte_pagos_oriflame['FechaFact'],dayfirst=True)
reporte_pagos_oriflame

,Cliente,Facturacion,FechaFact
0,Oriflame,34482.005,2021-01-11


# Invex

In [178]:
reporte_pagos_invex = pd.pivot_table(pagos_invex,index=['Cliente'],values=['AMT_TRAN','HONORARIO'],aggfunc=[np.sum,np.mean])
reporte_pagos_invex = pd.DataFrame(reporte_pagos_invex.to_records())
reporte_pagos_invex = reporte_pagos_invex.iloc[:,[0,1,4]]
reporte_pagos_invex.columns = ['Cliente','Pago','Honorario']
reporte_pagos_invex['Facturacion'] = reporte_pagos_invex['Pago']*reporte_pagos_invex['Honorario']
reporte_pagos_invex = reporte_pagos_invex.iloc[:,[0,3]]
reporte_pagos_invex['FechaFact'] = max(pagos_invex['TRAN_POST_DT'])
reporte_pagos_invex['FechaFact'] = pd.to_datetime(reporte_pagos_invex['FechaFact'],dayfirst=True)
reporte_pagos_invex

,Cliente,Facturacion,FechaFact
0,Invex,462644.114,2021-01-15


# Arabela

In [179]:
reporte_pagos_arabela = pd.pivot_table(pagos_arabela,index=['Cliente'],values=['factura'],aggfunc=np.sum)
reporte_pagos_arabela = pd.DataFrame(reporte_pagos_arabela.to_records())
reporte_pagos_arabela.columns = ['Cliente','Facturacion']
reporte_pagos_arabela['FechaFact'] = max(pagos_arabela['FechaEntrega'])
reporte_pagos_arabela['FechaFact'] = pd.to_datetime(reporte_pagos_arabela['FechaFact'],dayfirst=True)
reporte_pagos_arabela

,Cliente,Facturacion,FechaFact
0,Arabela,30832.8702,2021-01-12


# Union sucursal

In [180]:
pdList = [reporte_pagos_arabela,reporte_pagos_invex,reporte_pagos_oriflame]
final_torreon = pd.concat(pdList,ignore_index=True)
final_torreon = pd.merge(final_torreon,metas,on=['Cliente'],how='left')
final_torreon['Avance%'] = final_torreon['Facturacion']/final_torreon['Meta']
final_torreon.insert(loc=0,column='Sucursal',value='Torreon')
final_torreon

,Sucursal,Cliente,Facturacion,FechaFact,Meta,Avance%
0,Torreon,Arabela,30832.8702,2021-01-12,95000.0,0.324557
1,Torreon,Invex,462644.1140,2021-01-15,NaN,NaN
2,Torreon,Oriflame,34482.0050,2021-01-11,65000.0,0.530492


# Naucalpan

In [181]:
meta_naucalpan = pd.DataFrame([['Liverpool', 30000],['Liverpool Cof', 300000],['Liverpool Nacional', 550000],['Bradesco',750000],['Credifiel',100000]])
meta_naucalpan.columns = ['Cliente','Meta']

In [182]:
#Lectura pagos Bradesco
pagos_brad = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Bradesco/Pagos/Acumulado de pagos enero de 2021_1.xlsx')
pagos_brad = pagos_brad.iloc[:,0:9]
pagos_brad = pagos_brad.drop_duplicates()
pagos_brad2 = pd.pivot_table(pagos_brad,index=['NUMERO_TARJETA','FECHA_PAGO','TIPO_ASIGANCION'],values=['MONTO'],aggfunc=np.sum)
pagos_brad2 = pd.DataFrame(pagos_brad2.to_records())
pagos_brad2 = pagos_brad2.loc[pagos_brad2['FECHA_PAGO']>=mes3]

In [183]:
#Lectura pagos Liverpool
sheet_liv = ['SA002REM','SA001REM','DKN04REM','DKN02REM','DKN00REM']
pagos_liv = pd.DataFrame()
for i in sheet_liv:
    pagos_liv_new = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Liverpool/Pagos/PAGOS ENERO2021.xlsx',sheet_name = str(i))
    pagos_liv = pd.concat([pagos_liv,pagos_liv_new],ignore_index=True)
pagos_liv = pagos_liv.drop_duplicates()
pagos_liv = pagos_liv.iloc[:,0:6]
pagos_liv.columns = ['Id','Cuenta','Fecha Prom','Fecha Pago','Monto','Posteo']
pagos_liv['Fecha Pago'] = pd.to_datetime(pagos_liv['Fecha Pago'],dayfirst=True)
pagos_liv2 = pd.pivot_table(pagos_liv,index=['Id','Cuenta','Fecha Pago','Fecha Prom'],values=['Monto'],aggfunc=np.sum)
pagos_liv2 = pd.DataFrame(pagos_liv2.to_records())
pagos_liv2 = pagos_liv2.loc[pagos_liv2['Fecha Pago']>=mes3]
pagos_liv2['Cliente2'] = 'Liverpool'

In [184]:
#Lectura pagos Credifiel
sheet_cred = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Pagos/')
pagos_cred = pd.DataFrame()
for i in sheet_cred:
    pagos_cred_new = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Pagos/'+i+'')
    pagos_cred = pd.concat([pagos_cred,pagos_cred_new],ignore_index=True)
pagos_cred = pagos_cred.iloc[:,0:5]
pagos_cred = pagos_cred.drop_duplicates()
pagos_cred2 = pd.pivot_table(pagos_cred,index=['ID','FECHA'],values=['IMPORTE'],aggfunc=np.sum)
pagos_cred2 = pd.DataFrame(pagos_cred2.to_records())

In [185]:
#Hacemos la consulta de la asignacion
asignacion_cred = pd.read_excel('/home/estadistico/Documents/Erick/Credifiel/Asignacion/Asignacion Credifiel '+mes2+'.xlsx')
asignacion_cred = asignacion_cred.drop_duplicates(subset=['CREDITO','SALDO TOTAL REAL'])
asignacion_cred['CREDITO'] = pd.to_numeric(asignacion_cred['CREDITO'])
asignacion_cred = asignacion_cred[['CREDITO','ARANCEL']]

In [186]:
#Lectura pagos Banco Azteca Recovery
sheet_baz_recovery = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/Recovery/')
pagos_baz_recovery = pd.DataFrame()
for k in sheet_baz_recovery:
    pagos_baz_recovery_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/Recovery/'+k+'',sep='|')
    pagos_baz_recovery = pd.concat([pagos_baz_recovery,pagos_baz_recovery_new],ignore_index=True)
pagos_baz_recovery['Cartera'] = 'Recovery'
pagos_baz_recovery['DIA'] = pd.to_datetime(pagos_baz_recovery['DIA'],dayfirst=True)
pagos_baz_recovery = pagos_baz_recovery.loc[pagos_baz_recovery['DIA']>=mes3]
pagos_baz_recovery = pagos_baz_recovery.loc[pagos_baz_recovery['DIA']>=mes3]
pagos_baz_recovery['credito'] = pagos_baz_recovery['PAIS'].map(str)+'-'+pagos_baz_recovery['CANAL'].map(str)+'-'+pagos_baz_recovery['SUCURSAL'].map(str)+'-'+pagos_baz_recovery['FOLIO'].map(str)
pagos_baz_recovery = pagos_baz_recovery.drop_duplicates()

In [187]:
#Lectura pagos Banco Azteca 26-39
sheet_baz_26 = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/26-39/')
pagos_baz_26 = pd.DataFrame()
for j in sheet_baz_26:
    pagos_baz_26_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/26-39/'+j+'',sep='|')
    pagos_baz_26 = pd.concat([pagos_baz_26,pagos_baz_26_new],ignore_index=True)
pagos_baz_26['cartera'] = '26-39'
pagos_baz_26['DIA'] = pd.to_datetime(pagos_baz_26['DIA'],dayfirst=True)
pagos_baz_26 = pagos_baz_26.loc[pagos_baz_26['DIA']>=mes3]
pagos_baz_26['credito'] = pagos_baz_26['PAIS'].map(str)+'-'+pagos_baz_26['CANAL'].map(str)+'-'+pagos_baz_26['SUCURSAL'].map(str)+'-'+pagos_baz_26['FOLIO'].map(str)
pagos_baz_26 = pagos_baz_26.drop_duplicates()

# Liverpool

In [188]:
#Hacemos la asignacion de cada una de las metas para cambiar de arancel sin tomar el 0 en cuenta
cof, arancel_cof,arancel_cof2 = 300000, .5, .5
sub, arancel_sub, arancel_sub2 = 300000, .15, .2
subdic, arancel_subdic, arancel_subdic2 = 350000, .16, .2
n2, arancel_n2, arancel_n22 = 650000, .4, .5
pren, arancel_pren, arancel_pren2 = 300000, .28, .28

In [189]:
#Asignamos los aranceles de liverpool
aranceles = [('DKN00REM','Liverpool'),('DKN02REM','Liverpool Nacional'),('DKN04REM','Liverpool Cof'),('SA001REM','Liverpool'),('SA002REM','Liverpool')]
aranceles = pd.DataFrame(aranceles)
aranceles.columns = ['Id','Cliente']

In [190]:
#Obtenemos los pagos totales por division
pagos_totales_liv = pd.pivot_table(pagos_liv,index=['Id'],values=['Monto'],aggfunc=[np.sum])
pagos_totales_liv = pd.DataFrame(pagos_totales_liv.to_records())
#pagos_totales_liv = pagos_totales_liv2.iloc[:,[0,2,1]]
pagos_totales_liv.columns = ['Division','SumaPagos']
pagos_totales_liv.fillna(0,inplace=True)
pagos_totales_liv

,Division,SumaPagos
0,DKN00REM,399556.00
1,DKN02REM,357302.99
2,DKN04REM,180155.37
3,SA001REM,153899.98
4,SA002REM,16701.00


In [191]:
DKN00REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='DKN00REM', 'SumaPagos'].item()
DKN02REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='DKN02REM', 'SumaPagos'].item()
DKN04REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='DKN04REM', 'SumaPagos'].item()
SA001REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='SA001REM', 'SumaPagos'].item()
SA002REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='SA002REM', 'SumaPagos'].item()

In [192]:
def arancel_liv(row):
    if row['Id'] == 'DKN00REM' :
        return np.where((DKN00REM>0) & (DKN00REM<=pren/arancel_pren),arancel_pren,arancel_pren2)
    elif row['Id'] == 'DKN02REM':
        return np.where((DKN02REM>0) & (DKN02REM<=n2/arancel_n2),arancel_n2,arancel_n22)
    elif row['Id'] == 'DKN04REM':
        return np.where((DKN04REM>0) & (DKN04REM<=cof/arancel_cof),arancel_cof,arancel_cof2)
    elif row['Id'] == 'SA001REM':
        return np.where((SA001REM>0) & (SA001REM<=sub/arancel_sub),arancel_sub,arancel_sub2)
    elif row['Id'] == 'SA002REM':
        return np.where((SA002REM>0) & (SA002REM<=subdic/arancel_subdic),arancel_subdic,arancel_subdic2)
    else:
        return 0


In [193]:
pagos_liv['Arancel'] = pagos_liv.apply(arancel_liv, axis=1)

#Agregamos aranceles al consolidado
pagos_liv2 = pd.merge(pagos_liv,aranceles,how='left',on='Id')
pagos_liv2['PagoArancel'] = [x*y for x,y in zip(pagos_liv2['Monto'],pagos_liv2['Arancel'])]
pagos_liv2['Fecha Pago'] = pd.to_datetime(pagos_liv2['Fecha Pago'],dayfirst=True)
pagos_liv2['Cliente2'] = 'Liverpool'
pagos_liv2.head(1)

,Id,Cuenta,Fecha Prom,Fecha Pago,Monto,Posteo,Arancel,Cliente,PagoArancel,Cliente2
0,SA002REM,4050670000337528,07/01/2021,2021-01-07,500.0,5200000000009690,0.16,Liverpool,80.0,Liverpool


In [194]:
#Total pagos Liver registrados
pagos_totales_liv2 = pd.pivot_table(pagos_liv2,index=['Cliente2'],values=['PagoArancel'],aggfunc=np.sum,dropna=False)
pagos_totales_liv2 = pd.DataFrame(pagos_totales_liv2.to_records())
#pagos_totales_liv = pagos_totales_liv.iloc[:,[0,3,1,2]]
pagos_totales_liv2.columns = ['Cliente','Facturacion']
pagos_totales_liv2.fillna(0,inplace=True)
pagos_totales_liv2['Facturacion'] = pagos_totales_liv2['Facturacion'].round(2)
pagos_totales_liv2['FechaFact'] = max(pagos_liv2['Fecha Pago'])
pagos_totales_liv2['FechaFact'] = pd.to_datetime(pagos_totales_liv2['FechaFact'],dayfirst=True)
pagos_totales_liv2

,Cliente,Facturacion,FechaFact
0,Liverpool,370631.72,2021-01-17


In [195]:
def sig_arancel_liv(row):
    if row['Asignacion'] == 'DKN00REM' :
        return np.where((DKN00REM>0) & (DKN00REM<=pren/arancel_pren),round(pren/arancel_pren,1),0)
    elif row['Asignacion'] == 'DKN02REM':
        return np.where((DKN02REM>0) & (DKN02REM<=n2/arancel_n2),n2/arancel_n2,0)
    elif row['Asignacion'] == 'DKN04REM':
        return np.where((DKN04REM>0) & (DKN04REM<=cof/arancel_cof),cof/arancel_cof,0)
    elif row['Asignacion'] == 'SA001REM':
        return np.where((SA001REM>0) & (SA001REM<=sub/arancel_sub),sub/arancel_sub,0)
    elif row['Asignacion'] == 'SA002REM':
        return np.where((SA002REM>0) & (SA002REM<=subdic/arancel_subdic),subdic/arancel_subdic,0)
    
    else:
        return 0

In [196]:
pagos_liv_gerencia = pd.pivot_table(pagos_liv2,index=['Id'],values=['Monto'],aggfunc=[np.sum])
pagos_liv_gerencia = pd.DataFrame(pagos_liv_gerencia.to_records())
pagos_liv_gerencia.columns = ['Asignacion','Recuperado']
pagos_liv_gerencia.fillna(0,inplace=True)
pagos_liv_gerencia['FechaFact'] = max(pagos_liv2['Fecha Pago'])
pagos_liv_gerencia['SiguienteArancel'] = pagos_liv_gerencia.apply(sig_arancel_liv, axis=1)
pagos_liv_gerencia['RestSigArancel'] = pagos_liv_gerencia['SiguienteArancel']-pagos_liv_gerencia['Recuperado']
cols = ['Recuperado','SiguienteArancel','RestSigArancel']
pagos_liv_gerencia[cols] = pagos_liv_gerencia[cols].round(2)
pagos_liv_gerencia['RestSigArancel'] = np.where(pagos_liv_gerencia['RestSigArancel']<0,'Alcanzado',pagos_liv_gerencia['RestSigArancel'])
#pagos_liv_gerencia = pagos_liv_gerencia.iloc[:,[0,1,2,4]]
pagos_liv_gerencia.columns = ['Division','Recuperado','FechaFact','SiguienteArancel','RestSigArancel']
pagos_liv_gerencia

,Division,Recuperado,FechaFact,SiguienteArancel,RestSigArancel
0,DKN00REM,399556.00,2021-01-17,1071428.6,671873
1,DKN02REM,357302.99,2021-01-17,1625000.0,1.2677e+06
2,DKN04REM,180155.37,2021-01-17,600000.0,419845
3,SA001REM,153899.98,2021-01-17,2000000.0,1.8461e+06
4,SA002REM,16701.00,2021-01-17,2187500.0,2.1708e+06


# Bradesco

In [197]:
#Hacemos la asignacion de cada una de las metas para cambiar de arancel sin tomar el 0 en cuenta
c04pv1, arancel_4pv1 = 846436, .07
c04pv2, arancel_4pv2 = 1269654, .08
c04pv3, arancel_4pv3 = 1692873, .12
c05pv1, arancel_5pv1 = 729630, .09
c05pv2, arancel_5pv2 = 1094445, .10
c05pv3, arancel_5pv3 = 1459261, .14
c06pv1, arancel_6pv1 = 623524, .10
c06pv2, arancel_6pv2 = 935287, .11
c06pv3, arancel_6pv3 = 1247050, .16
a1, arancel_a1 = 264024, .27
a2, arancel_a2 = 396037, .30
a3, arancel_a3 = 528050, .33
b1, arancel_b1 = 214969, .25
b2, arancel_b2 = 396037, .28
b3, arancel_b3 = 528050, .31
c11, arancel_c11 = 1430989, .17
c12, arancel_c12 = 2146483, .20
c13, arancel_c13 = 2861978, .23
c21, arancel_c21 = 549549, .19
c22, arancel_c22 = 824323, .22
c23, arancel_c23 = 1099098, .25
c31, arancel_c31 = 237647, .21
c32, arancel_c32 = 356470, .24
c33, arancel_c33 = 475294, .27
c41, arancel_c41 = 109401, .23
c42, arancel_c42 = 164101, .26
c43, arancel_c43 = 218801, .29

In [198]:
#Obtenemos los pagos totales por division
pagos_totales_brad = pd.pivot_table(pagos_brad2,index=['TIPO_ASIGANCION'],values=['MONTO'],aggfunc=[np.sum,'count'],margins=True,dropna=False)
pagos_totales_brad = pd.DataFrame(pagos_totales_brad.to_records())
pagos_totales_brad = pagos_totales_brad.iloc[:,[0,2,1]]
pagos_totales_brad.columns = ['Division','NumeroPagos','SumaPagos']
pagos_totales_brad.fillna(0,inplace=True)

In [199]:
C04PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 4PV', 'SumaPagos'].item()
C05PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 5PV', 'SumaPagos'].item()
C06PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 6PV', 'SumaPagos'].item()
A = pagos_totales_brad.loc[pagos_totales_brad['Division']=='A', 'SumaPagos'].item()
C1 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C1', 'SumaPagos'].item()
C2 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C2', 'SumaPagos'].item()
C3 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C3', 'SumaPagos'].item()
C4 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C4', 'SumaPagos'].item()
B = pagos_totales_brad.loc[pagos_totales_brad['Division']=='B', 'SumaPagos'].item()

In [200]:
def sig_arancel(row):
    if row['Asignacion'] == 'C0 4PV':
        return np.where((C04PV>0) & (C04PV<=c04pv1),c04pv1,
                        np.where((C04PV>c04pv1) & (C04PV<=c04pv2),c04pv2,c04pv3))
    elif row['Asignacion'] == 'C0 5PV':
        return np.where((C05PV>0) & (C05PV<=c05pv1),c05pv1,
                        np.where((C05PV>c05pv1) & (C05PV<=c05pv2),c05pv2,c05pv3))
    elif row['Asignacion'] == 'C0 6PV':
        return np.where((C06PV>0) & (C06PV<=c06pv1),c06pv1,
                        np.where((C06PV>c06pv1) & (C06PV<=c06pv2),c06pv2,c06pv3))
    elif row['Asignacion'] == 'A':
        return np.where((A>0) & (A<=a1),a1,
                        np.where((A>a1) & (A<=a2),a2,a3))
    elif row['Asignacion'] == 'B':
        return np.where((B>0) & (B<=b1),b1,
                        np.where((B>b1) & (B<=b2),b2,b3))
    elif row['Asignacion'] == 'C1':
        return np.where((C1>0) & (C1<=c11),c11,
                        np.where((C1>c11) & (C1<=c12),c12,c13))
    elif row['Asignacion'] == 'C2':
        return np.where((C2>0) & (C2<=c21),c21,
                        np.where((C2>c21) & (C2<=c22),c22,c23))
    elif row['Asignacion'] == 'C3':
        return np.where((C3>0) & (C3<=c31),c31,
                        np.where((C3>c31) & (C3<=c32),c32,c33))
    elif row['Asignacion'] == 'C4':
        return np.where((C4>0) & (C4<=c41),c41,
                        np.where((C4>c41) & (C4<=c42),c42,c43))
    
    else:
        return 0

In [201]:
def func(row):
    if row['TIPO_ASIGANCION'] == 'C0 4PV':
        return np.where((C04PV>0) & (C04PV<=c04pv1),arancel_4pv1,
                        np.where((C04PV>c04pv1) & (C04PV<=c04pv2),arancel_4pv2,
                                 np.where((C04PV>c04pv2) & (C04PV<=c04pv3),arancel_4pv3,arancel_4pv3+.04)))
    elif row['TIPO_ASIGANCION'] == 'C0 5PV':
        return np.where((C05PV>0) & (C05PV<=c05pv1),arancel_5pv1,
                        np.where((C05PV>c05pv1) & (C05PV<=c05pv2),arancel_5pv2,
                                 np.where((C05PV>c05pv2) & (C05PV<=c05pv3),arancel_5pv3,arancel_5pv3+.04)))
    elif row['TIPO_ASIGANCION'] == 'C0 6PV':
        return np.where((C06PV>0) & (C06PV<=c06pv1),arancel_6pv1,
                        np.where((C06PV>c06pv1) & (C06PV<=c06pv2),arancel_6pv2,
                                 np.where((C06PV>c06pv2) & (C06PV<=c06pv3),arancel_6pv3,arancel_6pv3+.04)))
    elif row['TIPO_ASIGANCION'] == 'A':
        return np.where((A>0) & (A<=a1),arancel_a1,
                        np.where((A>a1) & (A<=a2),arancel_a2,
                                 np.where((A>a2) & (A<=a3),arancel_a3,arancel_a3+.04)))
    elif row['TIPO_ASIGANCION'] == 'B':
        return np.where((B>0) & (B<=b1),arancel_b1,
                        np.where((B>b1) & (B<=b2),arancel_b2,
                                 np.where((B>b2) & (B<=b3),arancel_b3,arancel_b3+.04)))
    elif row['TIPO_ASIGANCION'] == 'C1':
        return np.where((C1>0) & (C1<=c11),arancel_c11,
                        np.where((C1>c11) & (C1<=c12),arancel_c12,
                                 np.where((C1>c12) & (C1<=c13),arancel_c13,arancel_c13+.04)))
    elif row['TIPO_ASIGANCION'] == 'C2':
        return np.where((C2>0) & (C2<=c21),arancel_c21,
                        np.where((C2>c21) & (C2<=c22),arancel_c22,
                                 np.where((C2>c22) & (C2<=c23),arancel_c23,arancel_c23+.04)))
    elif row['TIPO_ASIGANCION'] == 'C3':
        return np.where((C3>0) & (C3<=c31),arancel_c31,
                        np.where((C3>c31) & (C3<=c32),arancel_c32,
                                 np.where((C3>c32) & (C3<=c33),arancel_c33,arancel_c33+.04)))
    elif row['TIPO_ASIGANCION'] == 'C4':
        return np.where((C4>0) & (C4<=c41),arancel_c41,
                        np.where((C4>c41) & (C4<=c42),arancel_c42,
                                 np.where((C4>c42) & (C4<=c43),arancel_c43,arancel_c43+.04)))
    
    else:
        return 0


In [202]:
pagos_brad2['Arancel'] = pagos_brad2.apply(func, axis=1)
pagos_brad2['Facturacion'] = [x*y for x,y in zip(pagos_brad2['MONTO'],pagos_brad2['Arancel'])]
pagos_brad2['Cliente'] = 'Bradesco'

In [203]:
pagos_totales_brad2 = pd.pivot_table(pagos_brad2,index=['Cliente'],values=['Facturacion'],aggfunc=[np.sum],dropna=False)
pagos_totales_brad2 = pd.DataFrame(pagos_totales_brad2.to_records())
pagos_totales_brad2.columns = ['Cliente','Facturacion']
pagos_totales_brad2.fillna(0,inplace=True)
pagos_totales_brad2['FechaFact'] = max(pagos_brad2['FECHA_PAGO'])
pagos_totales_brad2

,Cliente,Facturacion,FechaFact
0,Bradesco,137398.6074,2021-01-15


In [204]:
pagos_brad2.head(1)

,NUMERO_TARJETA,FECHA_PAGO,TIPO_ASIGANCION,MONTO,Arancel,Facturacion,Cliente
0,2869000540380975,2021-01-15,C4,1562.0,0.23,359.26,Bradesco


In [205]:
pagos_brad_gerencia = pd.pivot_table(pagos_brad2,index=['TIPO_ASIGANCION'],values=['MONTO'],aggfunc=[np.sum])
pagos_brad_gerencia = pd.DataFrame(pagos_brad_gerencia.to_records())
pagos_brad_gerencia.columns = ['Asignacion','Recuperado']
pagos_brad_gerencia.fillna(0,inplace=True)
pagos_brad_gerencia['FechaFact'] = max(pagos_brad2['FECHA_PAGO'])
pagos_brad_gerencia['SiguienteArancel'] = pagos_brad_gerencia.apply(sig_arancel, axis=1)
pagos_brad_gerencia['RestSigArancel'] = pagos_brad_gerencia['SiguienteArancel']-pagos_brad_gerencia['Recuperado']
cols = ['Recuperado','SiguienteArancel','RestSigArancel']
pagos_brad_gerencia[cols] = pagos_brad_gerencia[cols].round(2)
#pagos_brad_gerencia = pagos_brad_gerencia.iloc[:,[0,1,2,4]]
pagos_brad_gerencia.columns = ['Division','Recuperado','FechaFact','SiguienteArancel','RestSigArancel']
pagos_brad_gerencia

,Division,Recuperado,FechaFact,SiguienteArancel,RestSigArancel
0,A,33962.00,2021-01-15,264024,230062
1,B,25364.00,2021-01-15,214969,189605
2,C0 4PV,325377.32,2021-01-15,846436,521059
3,C0 5PV,132900.06,2021-01-15,729630,596730
4,C0 6PV,117564.50,2021-01-15,623524,505960
5,C1,183434.00,2021-01-15,1430989,1.24756e+06
6,C2,139184.84,2021-01-15,549549,410364
7,C3,66521.00,2021-01-15,237647,171126
8,C4,16503.00,2021-01-15,109401,92898


# Credifiel

In [206]:
#Hacemos el consolidado de la asignacion y promesado
consolidado_pagos_cred = pd.merge(pagos_cred2,asignacion_cred,how='left',left_on=['ID'],right_on=['CREDITO'])
consolidado_pagos_cred.drop('CREDITO',1,inplace=True)
consolidado_pagos_cred['ARANCEL'] = np.where(consolidado_pagos_cred['ARANCEL'].isna(),np.mean(consolidado_pagos_cred['ARANCEL']),consolidado_pagos_cred['ARANCEL'])
consolidado_pagos_cred['FACTURACION'] = consolidado_pagos_cred['IMPORTE']*consolidado_pagos_cred['ARANCEL']
consolidado_pagos_cred['CLIENTE'] = 'Credifiel'
consolidado_pagos_cred.head(1)

,ID,FECHA,IMPORTE,ARANCEL,FACTURACION,CLIENTE
0,76245,2021-01-11,30000.0,0.201471,6044.117647,Credifiel


In [207]:
pagos_totales_cred = pd.pivot_table(consolidado_pagos_cred,index=['CLIENTE'],values=['FACTURACION'],aggfunc=[np.sum],dropna=False)
pagos_totales_cred = pd.DataFrame(pagos_totales_cred.to_records())
pagos_totales_cred.columns = ['Cliente','Facturacion']
pagos_totales_cred.fillna(0,inplace=True)
pagos_totales_cred['FechaFact'] = max(consolidado_pagos_cred['FECHA'])
pagos_totales_cred

,Cliente,Facturacion,FechaFact
0,Credifiel,31515.158588,2021-01-14


In [208]:
pagos_cred_gerencia = pd.pivot_table(consolidado_pagos_cred,index=['CLIENTE'],values=['IMPORTE'],aggfunc=np.sum)
pagos_cred_gerencia = pd.DataFrame(pagos_cred_gerencia.to_records())
pagos_cred_gerencia[' FechaFact'] = max(consolidado_pagos_cred['FECHA'])
pagos_cred_gerencia.columns = ['Division','Recuperado','FechaFact']
pagos_cred_gerencia

,Division,Recuperado,FechaFact
0,Credifiel,184476.96,2021-01-14


# Banco Azteca

In [209]:
# pagos_baz = pd.concat([pagos_baz_26,pagos_baz_recovery],ignore_index=True)
# pagos_baz = pagos_baz.loc[pagos_baz['CARGO AUTOMATICO']=='NO']
# pagos_baz['Recuperado'] = pagos_baz['RECUPERACION DE CAPITAL']+pagos_baz['RECUPERACION DE MORATORIOS']
# pagos_baz['Cliente'] = 'Banco Azteca'

In [210]:
# pagos_totales_baz = pd.pivot_table(fact_baz,index=['Cliente'],values=['Total Recuperado','Total Comisión'],aggfunc=[np.sum],dropna=False)
# pagos_totales_baz = pd.DataFrame(pagos_totales_baz.to_records())
# pagos_totales_baz.columns = ['Cliente','Recuperado','SumaPagos']
# porcentaje_prom_arancel_baz = pagos_totales_baz['Recuperado']/pagos_totales_baz['SumaPagos']
# porcentaje_prom_arancel_baz

In [211]:
# pagos_baz['Facturacion'] = [x*porcentaje_prom_arancel_baz for x in pagos_baz['Recuperado']]
# pagos_fact_baz = pd.pivot_table(pagos_baz,index=['Cliente'],values=['Facturacion'],aggfunc=[np.sum])
# pagos_fact_baz = pd.DataFrame(pagos_fact_baz.to_records())
# pagos_fact_baz.columns = ['Cliente','Facturacion']
# pagos_fact_baz['FechaFact'] = max(pagos_baz['DIA'])
# pagos_fact_baz

# Union Naucalpan

In [212]:
pdList = [pagos_totales_liv2,pagos_totales_brad2,pagos_totales_cred]
final_naucalpan = pd.concat(pdList,ignore_index=True)
final_naucalpan = pd.merge(final_naucalpan,meta_naucalpan,on=['Cliente'],how='left')
final_naucalpan['Avance%'] = final_naucalpan['Facturacion']/final_naucalpan['Meta']
final_naucalpan.insert(loc=0,column='Sucursal',value='Naucalpan')
final_naucalpan

,Sucursal,Cliente,Facturacion,FechaFact,Meta,Avance%
0,Naucalpan,Liverpool,370631.720000,2021-01-17,1050000,0.352983
1,Naucalpan,Bradesco,137398.607400,2021-01-15,750000,0.183198
2,Naucalpan,Credifiel,31515.158588,2021-01-14,100000,0.315152


# Guatemala

In [213]:
final_guatemala = pd.DataFrame([['Avon G',69940,250000],['Arabela G',808,5100],['Banco Azteca G',6047,52000],['Banco Credomatic',2222,17000],['Belcorp',2010,5000],['Banco Azteca R',37152,115000]])
final_guatemala.columns = ['Cliente','Facturacion','Meta']
final_guatemala.insert(loc=0,column='Sucursal',value='Guatemala')

In [214]:
final_guatemala['Facturacion'] = final_guatemala['Facturacion']*quetzal
final_guatemala['Meta'] = final_guatemala['Meta']*quetzal
final_guatemala['FechaFact'] = ayer

# Panama

In [215]:
final_panama = pd.DataFrame([['Cable Onda',1436,2800],['Jamar',2500,8000],['Banco Azteca',4419,4000],['Adel Panama',893,3000],['Cable Onda Act',627,2000],['Banco Azteca Act',31000,34000],['Avon',485,2000],['Avon Renta',1250,1250],['Rodelag',0,3500]])
final_panama.columns = ['Cliente','Facturacion','Meta']
final_panama.insert(loc=0,column='Sucursal',value='Panama')

In [216]:
final_panama['Facturacion'] = final_panama['Facturacion']*dolar
final_panama['Meta'] = final_panama['Meta']*dolar
final_panama['FechaFact'] = ayer
final_panama

,Sucursal,Cliente,Facturacion,Meta,FechaFact
0,Panama,Cable Onda,28432.8,55440.0,2021-01-18
1,Panama,Jamar,49500.0,158400.0,2021-01-18
2,Panama,Banco Azteca,87496.2,79200.0,2021-01-18
3,Panama,Adel Panama,17681.4,59400.0,2021-01-18
4,Panama,Cable Onda Act,12414.6,39600.0,2021-01-18
5,Panama,Banco Azteca Act,613800.0,673200.0,2021-01-18
6,Panama,Avon,9603.0,39600.0,2021-01-18
7,Panama,Avon Renta,24750.0,24750.0,2021-01-18
8,Panama,Rodelag,0.0,69300.0,2021-01-18


# Torreon   

In [217]:
final_torreon = pd.DataFrame([['Avon',42681,990000],['Oriflame',22307,65000],['Invex',462644,1095000],['Arabela',44315,95000],['Cimaco',547,3500],['Cristian Lay',0,6000]])
final_torreon.columns = ['Cliente','Facturacion','Meta']
final_torreon.insert(loc=0,column='Sucursal',value='Torreon')
final_torreon['FechaFact'] = ayer
final_torreon

,Sucursal,Cliente,Facturacion,Meta,FechaFact
0,Torreon,Avon,42681,990000,2021-01-18
1,Torreon,Oriflame,22307,65000,2021-01-18
2,Torreon,Invex,462644,1095000,2021-01-18
3,Torreon,Arabela,44315,95000,2021-01-18
4,Torreon,Cimaco,547,3500,2021-01-18
5,Torreon,Cristian Lay,0,6000,2021-01-18
6,Torreon,Avon,485,2000,2021-01-18
7,Torreon,Avon Renta,1250,1250,2021-01-18
8,Torreon,Rodelag,0,3500,2021-01-18


# Union final

In [218]:
toadd_final = [final_naucalpan,final_torreon,final_guatemala,final_panama]
final_mexico = pd.concat(toadd_final,ignore_index=True)
final_mexico = pd.concat([final_mexico,final_guatemala],ignore_index=True)
final_mexico = pd.concat([final_mexico,final_panama],ignore_index=True)
final_mexico['Avance%'] = final_mexico['Facturacion']/final_mexico['Meta']

In [219]:
final_mexico['MetaDia'] = ((final_mexico['FechaFact'].dt.day/30)*final_mexico['Meta']).round()
final_mexico['TendenciaCierre'] = [(30/x.day)*y for x,y in zip(final_mexico['FechaFact'],final_mexico['Facturacion'])]
final_mexico['Semaforo'] = np.where(final_mexico['MetaDia']<=final_mexico['Facturacion'],'verde',
                                    np.where((final_mexico['MetaDia']*.75<=final_mexico['Facturacion']) &                                                                                    (final_mexico['MetaDia']>final_mexico  ['Facturacion']),'amarillo','rojo'))
final_mexico
fecha_fact = datetime.strftime(max(final_mexico['FechaFact']),'%d-%m-%Y')

,Sucursal,Cliente,Facturacion,FechaFact,Meta,Avance%,MetaDia,TendenciaCierre,Semaforo
0,Naucalpan,Liverpool,370631.720000,2021-01-17,1050000.0,0.352983,595000.0,6.540560e+05,rojo
1,Naucalpan,Bradesco,137398.607400,2021-01-15,750000.0,0.183198,375000.0,2.747972e+05,rojo
2,Naucalpan,Credifiel,31515.158588,2021-01-14,100000.0,0.315152,46667.0,6.753248e+04,rojo
3,Torreon,Avon,42681.000000,2021-01-18,990000.0,0.043112,594000.0,7.113500e+04,rojo
4,Torreon,Oriflame,22307.000000,2021-01-18,65000.0,0.343185,39000.0,3.717833e+04,rojo
5,Torreon,Invex,462644.000000,2021-01-18,1095000.0,0.422506,657000.0,7.710733e+05,rojo
6,Torreon,Arabela,44315.000000,2021-01-18,95000.0,0.466474,57000.0,7.385833e+04,amarillo
7,Torreon,Cimaco,547.000000,2021-01-18,3500.0,0.156286,2100.0,9.116667e+02,rojo
8,Torreon,Cristian Lay,0.000000,2021-01-18,6000.0,0.000000,3600.0,0.000000e+00,rojo
9,Torreon,Avon,485.000000,2021-01-18,2000.0,0.242500,1200.0,8.083333e+02,rojo


In [220]:
final_sucursal = pd.pivot_table(final_mexico,index=['Sucursal'],values=['Meta','Facturacion','FechaFact'],aggfunc=[np.sum,np.max])
final_sucursal = pd.DataFrame(final_sucursal.to_records())
final_sucursal = final_sucursal.iloc[:,[0,1,2,4]]
final_sucursal.columns = ['Sucursal','Facturacion','Meta','FechaFact']
final_sucursal['Avance'] = final_sucursal['Facturacion']/final_sucursal['Meta']
final_sucursal['MetaDia'] = ((final_sucursal['FechaFact'].dt.day/30)*final_sucursal['Meta']).round()
final_sucursal['Semaforo'] = np.where(final_sucursal['MetaDia']<=final_sucursal['Facturacion'],'Verde',
                                    np.where((final_sucursal['MetaDia']*.75<=final_sucursal['Facturacion']) &(final_sucursal['MetaDia']>final_sucursal['Facturacion']),'Amarillo','Rojo'))
final_sucursal

,Sucursal,Facturacion,Meta,FechaFact,Avance,MetaDia,Semaforo
0,Guatemala,300174.660000,1128014.0,2021-01-18,0.266109,676808.0,Rojo
1,Naucalpan,539545.485988,1900000.0,2021-01-17,0.283971,1076667.0,Rojo
2,Panama,843678.000000,1198890.0,2021-01-18,0.703716,719334.0,Verde
3,Torreon,574229.000000,2261250.0,2021-01-18,0.253943,1356750.0,Rojo


In [221]:
final_bradlivcred = pd.concat([pagos_liv_gerencia,pagos_brad_gerencia],ignore_index=True)
final_bradlivcred = pd.concat([final_bradlivcred,pagos_cred_gerencia],ignore_index=True)
final_bradlivcred

,Division,Recuperado,FechaFact,SiguienteArancel,RestSigArancel
0,DKN00REM,399556.00,2021-01-17,1071428.6,671873
1,DKN02REM,357302.99,2021-01-17,1625000.0,1.2677e+06
2,DKN04REM,180155.37,2021-01-17,600000.0,419845
3,SA001REM,153899.98,2021-01-17,2000000.0,1.8461e+06
4,SA002REM,16701.00,2021-01-17,2187500.0,2.1708e+06
5,A,33962.00,2021-01-15,264024,230062
6,B,25364.00,2021-01-15,214969,189605
7,C0 4PV,325377.32,2021-01-15,846436,521059
8,C0 5PV,132900.06,2021-01-15,729630,596730
9,C0 6PV,117564.50,2021-01-15,623524,505960


In [222]:
reporte_pagos_nauc = pd.read_csv(''+ruta_pagos+'/'+mes2+'/Reportes diarios/Reporte sucursal/Reporte sucursal '+today2+'.csv')

In [223]:
final_bradlivcred2 = final_bradlivcred.iloc[:,[0,3,4]]
final_bradlivcred3 = pd.merge(reporte_pagos_nauc,final_bradlivcred2,on=['Division'],how='left')
cols = ['SumaFacturacion','TendenciaCierre1']
final_bradlivcred3[cols] = final_bradlivcred3[cols].round(2)
final_bradlivcred3

,Pais,Sucursal,Cliente,Division,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact,NumeroPromesas,SumaPromesas,PromesasRestantes,TendenciaCierre1,RecuperadoNoPromesa,PagosNoPromesa,SiguienteArancel,RestSigArancel
0,Mexico,Naucalpan,Liverpool,DKN00REM,103,399556.00,111875.68,2021-01-17,179,749282.74,192472.87,197427.67,67649.00,21,1071428.6,671873
1,Mexico,Naucalpan,Liverpool,DKN02REM,97,357302.99,142921.20,2021-01-17,152,669093.44,151329.00,252213.88,64820.99,36,1625000.0,1.2677e+06
2,Mexico,Naucalpan,Liverpool,DKN04REM,50,180155.37,90077.68,2021-01-17,78,261983.37,72174.00,158960.62,28053.00,8,600000.0,419845
3,Mexico,Naucalpan,Liverpool,SA001REM,107,153899.98,23085.00,2021-01-17,170,272277.24,81950.10,40738.23,25609.83,28,2000000.0,1.8461e+06
4,Mexico,Naucalpan,Liverpool,SA002REM,9,16701.00,2672.16,2021-01-17,10,21957.00,0.00,4715.58,4305.00,4,2187500.0,2.1708e+06
5,Mexico,Naucalpan,Bradesco,A,16,33962.00,9169.74,2021-01-15,33,61077.00,7000.00,18339.48,3720.00,1,264024,230062
6,Mexico,Naucalpan,Bradesco,B,5,25364.00,6341.00,2021-01-15,6,10201.00,0.00,12682.00,21817.00,2,214969,189605
7,Mexico,Naucalpan,Bradesco,C0 4PV,224,325377.32,22776.41,2021-01-15,285,458991.09,38314.00,45552.82,102219.32,85,846436,521059
8,Mexico,Naucalpan,Bradesco,C0 5PV,106,132900.06,11961.01,2021-01-15,129,231211.00,21077.00,23922.01,56335.03,48,729630,596730
9,Mexico,Naucalpan,Bradesco,C0 6PV,76,117564.50,11756.45,2021-01-15,101,187317.00,5993.00,23512.90,36487.50,31,623524,505960


In [224]:
writer = pd.ExcelWriter(''+ruta_guardar+'/Avance Pagos '+fecha_fact+'.xlsx',engine='xlsxwriter')
final_sucursal.to_excel(writer,'FactSucursal',index=False,header=True)
final_mexico.to_excel(writer,'FactCliente',index=False,header=True)
final_bradlivcred3.to_excel(writer,'GerEnrique',index=False,header=True)

writer.save()